<a href="https://colab.research.google.com/github/daisyKim12/Tensorflow_Study/blob/main/Practice_C4_sarcasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
####################################################
## [중요]
## 2023년 6월 1일 기준
## Google Colab에 설치된 텐서플로우(TensorFlow) 버전은 2.12 버전입니다.
## 시험에 제출 가능한 버전은 2.9.0 버전이기 때문에 버전을 다운그레이드 진행해야 합니다.
## 반드시 아래 코드를 실행하여 버전 다운그레이드 후 모델링을 진행하세요.
## 시험 볼 때는 현재 이 코드는 지워주셔야 합니다.
## 관련하여 궁금하신 점은 슬랙 커뮤니티에 질문 남겨 주세요.
####################################################
import urllib.request

url = 'https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/tfcert.py'
urllib.request.urlretrieve(url, 'tfcert.py')
%run tfcert.py

import tensorflow as tf
import tensorflow_datasets as tfds

print(f'설치 완료 후 TensorFlow 버전: {tf.__version__}')
print(f'설치 완료 후 TensorFlow Datasets 버전: {tfds.__version__}')

## 시험을 위한 버전 확인 ###########
## TensorFlow:          2.9.0 #
## TensorFlow Datasets: 4.6.0 #
###############################

설치환경: Google Colab
TensorFlow 시험환경을 구성중입니다. 잠시만 기다려 주세요.
(설치는 약 1~5분 정도 소요 됩니다)
[알림] TensorFlow 시험환경 구성이 완료 되었습니다.


/usr/local/lib/python3.10/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


설치 완료 후 TensorFlow 버전: 2.9.0
설치 완료 후 TensorFlow Datasets 버전: 4.6.0


In [ ]:
# ======================================================================
# There are 5 questions in this test with increasing difficulty from 1-5
# Please note that the weight of the grade for the question is relative
# to its difficulty. So your Category 1 question will score much less
# than your Category 5 question.
# ======================================================================
#
# NLP QUESTION
#
# For this task you will build a classifier for the sarcasm dataset
# The classifier should have a final layer with 1 neuron activated by sigmoid as shown
# It will be tested against a number of sentences that the network hasn't previously seen
# And you will be scored on whether sarcasm was correctly detected in those sentences

# =========== 합격 기준 가이드라인 공유 ============= #
# val_loss 기준에 맞춰 주시는 것이 훨씬 더 중요 #
# val_loss 보다 조금 높아도 상관없음. (언저리까지 OK) #
# =================================================== #
# 문제명: Category 4 - sarcasm
# val_loss: 0.3650
# val_acc: 0.83
# =================================================== #
# =================================================== #


import json
import tensorflow as tf
import numpy as np
import urllib

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Flatten, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint


def solution_model():
  url = 'https://storage.googleapis.com/download.tensorflow.org/data/sarcasm.json'
  urllib.request.urlretrieve(url, 'sarcasm.json')

  # DO NOT CHANGE THIS CODE OR THE TESTS MAY NOT WORK
  vocab_size = 1000
  embedding_dim = 16
  max_length = 120
  trunc_type='post'
  padding_type='post'
  oov_tok = "<OOV>"
  training_size = 20000

  sentences = []
  labels = []

  # YOUR CODE HERE
  with open('sarcasm.json') as f:
    datas = json.load(f)

  for data in datas:
    sentences.append(data['headline'])
    labels.append(data['is_sarcastic'])

  training_size = 20000

  train_sentences = sentences[:training_size]
  train_labels = labels[:training_size]

  validation_sentences = sentences[training_size:]
  validation_labels = labels[training_size:]

  vocab_size = 1000
  oov_tok = "<OOV>"
  tokenizer = Tokenizer(num_words=vocab_size, oov_token = '<OOV>')
  tokenizer.fit_on_texts(train_sentences)

  train_sequences = tokenizer.texts_to_sequences(train_sentences)
  validation_sequences = tokenizer.texts_to_sequences(validation_sentences)

  max_length = 120
  trunc_type = 'post'
  padding_type = 'post'

  train_padded = pad_sequences(train_sequences, maxlen=max_length, truncating = trunc_type, padding = padding_type)
  validation_padded = pad_sequences(validation_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  train_labels = np.array(train_labels)
  validation_labels = np.array(validation_labels)

  embedding_dim = 16
  model = tf.keras.Sequential([
      Embedding(vocab_size, embedding_dim, input_length=max_length),
      Bidirectional(LSTM(64, return_sequences=True)),
      Bidirectional(LSTM(64)),
      Dense(256, activation='relu'),
      Dropout(0.5),
      Dense(128, activation='relu'),
      Dense(64, activation ='relu'),
      Dense(16, activation='relu'),

      # YOUR CODE HERE. KEEP THIS OUTPUT LAYER INTACT OR TESTS MAY FAIL
      tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

  checkpoint_path = 'my_checkpoint.ckpt'
  checkpoint = ModelCheckpoint(checkpoint_path,
                              save_weights_only=True,
                              save_best_only=True,
                              monitor='val_loss',
                              verbose=1)

  model.fit(train_padded, train_labels,
            validation_data=(validation_padded, validation_labels),
            callbacks=[checkpoint],
            epochs=10,
            verbose = 1)

  model.load_weights(checkpoint_path)

  return model


# Note that you'll need to save your model as a .h5 like this
# This .h5 will be uploaded to the testing infrastructure
# and a score will be returned to you
if __name__ == '__main__':
  model = solution_model()
  model.save("TF4-sarcasm.h5")


Epoch 1/10
624/625 [============================>.] - ETA: 0s - loss: 0.4607 - acc: 0.7622
Epoch 1: val_loss improved from inf to 0.39340, saving model to my_checkpoint.ckpt
625/625 [==============================] - 22s 26ms/step - loss: 0.4604 - acc: 0.7623 - val_loss: 0.3934 - val_acc: 0.8226
Epoch 2/10
623/625 [============================>.] - ETA: 0s - loss: 0.3491 - acc: 0.8427
Epoch 2: val_loss improved from 0.39340 to 0.37560, saving model to my_checkpoint.ckpt
625/625 [==============================] - 14s 22ms/step - loss: 0.3488 - acc: 0.8429 - val_loss: 0.3756 - val_acc: 0.8298
Epoch 3/10
623/625 [============================>.] - ETA: 0s - loss: 0.3234 - acc: 0.8578
Epoch 3: val_loss did not improve from 0.37560
625/625 [==============================] - 14s 22ms/step - loss: 0.3237 - acc: 0.8579 - val_loss: 0.3991 - val_acc: 0.8228
Epoch 4/10
623/625 [============================>.] - ETA: 0s - loss: 0.3094 - acc: 0.8639
Epoch 4: val_loss improved from 0.37560 to 0.36931